# Chiral solutions for Type-I Dirac seesaw

In [1]:
import pandas as pd
import numpy as np
import itertools
import sys
pd.set_option('display.max_colwidth',400)

## Load full solutions

In [30]:
ds=pd.read_json('https://github.com/restrepo/anomaly/raw/main/solutions.json')
ds.shape

(31211, 5)

Check that solutions are unique

In [3]:
ds['strz']=ds['solution'].astype(str)
ds.drop_duplicates('strz').shape

(31211, 6)

In [4]:
ds=ds.drop('strz',axis='columns')

Number of solutions with repeated charges

In [5]:
ds[ds['solution'].apply(lambda l: len(l)-len(set(l)))>0].shape[0]

16805

## Check phenomenological conditions
$$ 2l+\nu-m=0$$

In [6]:
#See anomalysolutions.ipynb for details
#from anomalytools import *

### Prepare functions to filter solutions with at least two sets of repeated charges

In [7]:
def multiple_repeated(ll):
    MR=False
    rp=[]
    for x in ll:
        if ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(rp))
    if len(rp)>=2:
        MR=True
    return MR

def extract_Dirac_and_Majorana(l,s):
    ll=l.copy()
    #Check first Dirac
    for xs  in itertools.combinations(l,2):
        #print('ini',xs,ll,xs[0] in ll,xs[1] in ll)
        if (xs[0] in ll  and xs[1] in ll)  and (sum(xs)+s==0 or sum(xs)-s==0):
            ll=[x for x in ll if x in ll and x not in xs]
        #print('ll',ll)
    #Check Majorana
    for x in ll:
        if 2*x+s==0 or 2*x-s==0:
            ll.remove(x)
    return ll

def best_higgs_singlet(l):
    lenmax=np.Inf
    best_sp=0
    best_spl=l
    sps=np.unique( (np.abs( list(2*np.array(l))+[sum(x) for x in itertools.combinations(l,2) ]) ) )
    for sp in sps:
        newl=extract_Dirac_and_Majorana(l,sp)
        if len(newl)<len(best_spl):
            best_spl=newl
            best_sp=sp
    return best_sp,best_spl

def tree_level(ll):
    '''
    Given a solution find set of repeated charges and assign to `l` and `ν`, e.g
       ll=[l,l,...,ν,ν,...]
    For each combination of repeated sets calculates:
       m=2l+ν
    If `m ≠ l ≠ ν` in ll, returns a `N → N+4` solution
       [m,m,m,-m-m,l,l,...,ν,ν,...]
    else, returns a `N → N+6` solution
       [m,m,m,-m,-m,-m,l,l,...,ν,ν,...]
    '''
    ms=[]
    rp=[]
    for x in ll:
        if ll.count(x)==1:
            ms.append(x)
        elif ll.count(x)>1:
            rp.append(x)
    rp=sorted(np.unique(rp))
    #print(rp,isinstance(rp,list))
    #Two or more repeated sets
    sltn=[]
    if len(rp)>=2:
        rps=[k  for k  in itertools.combinations(rp,2)]
        #print('rps1',rps)
        rps=rps+[tuple(reversed(k)) for k  in rps]
        #print('rps2',rps)
        for r in rps:
            #Loop on combinatories here, break whit the first solution
            l=r[0]
            ν=r[1]
            m=2*l+ν
            s=l-m
            massless=ms.copy()
            massless=massless+[x for x in rp if x not in r and x not in massless]
            #print('ini massless',massless)
            if m in ll and m not in rp:
                massless.remove(m)
                #Check Dirac and Majorana
                massless=extract_Dirac_and_Majorana(massless,s)
                slt={'[l,ν,s,m_in,m_out]':[l,ν,s,m,0],'massless':massless}
            else:
                slt={'[l,ν,s,m_in,m_out]':[l,ν,s,0,m],'massless':massless}
                
            if len(massless)>0:
                slt['sp'],slt['sp_massless']=best_higgs_singlet(massless)
            else:
                slt['sp'],slt['sp_massless']=(None,None)
            sltn.append(slt)
    return sltn

assert extract_Dirac_and_Majorana([2,6,4,3],8)==[3]
assert best_higgs_singlet([1,-11,9])==(2, [])

assert tree_level([1, -4, -4, 9, 9,7,7,7,-11])[0]['[l,ν,s,m_in,m_out]']==[-4, 7, -3, 0, -1]

assert tree_level([ -4, -4,-4, 9, 9,9])[0]['sp']==None

### Initialize filtered solutions

In [8]:
cl=ds.copy()
cl=cl[cl['solution'].apply(multiple_repeated)].reset_index(drop=True)

Find solutions that satisfy the condition

In [9]:
cl['tree_level']=cl['solution'].apply(tree_level)

Reorder the filtered solutions

In [10]:
cl['smax']=cl['solution'].apply(lambda l:np.abs(l).max())
cl=cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
               ).apply(len)>0].sort_values(['n','smax']).reset_index(drop=True)

cl['tree_level']=cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 or 
                                                   len(d.get('sp_massless'))==0 ] if isinstance(l,list) else None 
             )
cl=cl[cl.smax<=30].reset_index(drop=True)
cl[:1]

,l,k,solution,gcd,n,tree_level,smax
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[l,ν,s,m_in,m_out]': [-4, 1, 3, 0, -7], 'massless': [5], 'sp': 10, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -4, 3, 0, -2], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5


### Flag equivalent solutions

In [11]:
def characterize_solution(cl,i,j=0):
    if j%2==0:
        cl.loc[i,'D_msls']=j
    else:
        cl.loc[i,'M_msls']=j
    #if j==0:
    #    cl.loc[i,'M_msls']=j
        
    cl.loc[i,'rptd']=len( [ cl.loc[i,'solution'].count(x) for x in np.unique(cl.loc[i,'solution']) if cl.loc[i,'solution'].count(x)>1] )
    sl=[d.get('[l,ν,s,m_in,m_out]') for d in cl.loc[i,'tree_level'] if len(d.get('massless'))==j]
    if len(sl)>0:
        cl.loc[i,'nl']=cl.loc[i,'solution'].count(sl[0][0] )
        cl.loc[i,'nν'] =cl.loc[i,'solution'].count(sl[0][1] )
    if 0 in [l[3] for l in sl]:
        cl.loc[i,'nr']=3
    else:
        cl.loc[i,'nr']=2
    return cl

Example: massless solutions

In [12]:
for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if len(d.get('massless'))==0 ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
    cl=characterize_solution(cl,i,0)

In [13]:
cl.dropna()

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,nν,nr
179,"[3, -2, 3]","[-2, -1, -2, 4]","[4, 4, 4, -5, -9, -10, -10, 11, 11]",1,9,"[{'[l,ν,s,m_in,m_out]': [-10, 11, -1, -9, 0], 'massless': [], 'sp': None, 'sp_massless': None}]",11,0.0,3.0,2.0,2.0,2.0
212,"[3, 5, 3]","[-2, -1, -2, -3]","[2, -5, 6, 9, 9, -12, -12, -13, 16]",2,9,"[{'[l,ν,s,m_in,m_out]': [9, -12, 3, 6, 0], 'massless': [], 'sp': None, 'sp_massless': None}]",16,0.0,2.0,2.0,2.0,2.0
215,"[1, 0, 1]","[-3, -4, -1, -2]","[5, 6, -8, 9, 9, -12, -12, -14, 17]",3,9,"[{'[l,ν,s,m_in,m_out]': [9, -12, 3, 6, 0], 'massless': [], 'sp': None, 'sp_massless': None}]",17,0.0,2.0,2.0,2.0,2.0
269,"[-4, -1, 6]","[-5, -2, 3, -1]","[1, -5, 8, 12, 12, -16, -16, -17, 21]",3,9,"[{'[l,ν,s,m_in,m_out]': [-16, 12, 4, 0, -20], 'massless': [1, -5, 8, -17, 21], 'sp': 16, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [12, -16, 4, 8, 0], 'massless': [], 'sp': None, 'sp_massless': None}]",21,0.0,2.0,2.0,2.0,2.0
312,"[5, 7, 5]","[-2, 0, -2, -4]","[1, 4, -10, -15, -15, 20, 20, 21, -26]",4,9,"[{'[l,ν,s,m_in,m_out]': [-15, 20, -5, -10, 0], 'massless': [], 'sp': None, 'sp_massless': None}]",26,0.0,2.0,2.0,2.0,2.0


### Add solutions with massless fermions that get masses with some $S'$

In [14]:
for j in range(1,6):
    print(j)
    for i in cl[cl['tree_level'].apply(lambda l: [d for d in l if d.get('sp_massless') is not None and len(d.get('sp_massless'))==0  and len(d.get('massless'))==j ] if isinstance(l,list) else None 
             ).apply(len)>0].index:
        cl=characterize_solution(cl,i,j)

1
2
3
4
5


In [15]:
cl['D_msls']=cl['D_msls'].apply(lambda n: 10 if np.isnan(n) else n)
cl['M_msls']=cl['M_msls'].apply(lambda n: 10 if np.isnan(n) else n)
for k  in ['D_msls','M_msls','nl','nν','nr','rptd']:
    cl[k]=cl[k].astype(int)

In [16]:
cl=cl.sort_values(['n','smax','D_msls','rptd','nl','nν','nr']).reset_index(drop=True)

## Build the final table dropping out equivalent solutions 

In [19]:
fl=pd.DataFrame()

In [20]:
fl=fl.append( cl[( (cl['n']==9) & (cl['D_msls']==0) )].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [21]:
cl[ cl['n']==6 ]

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,nν,nr,M_msls
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[l,ν,s,m_in,m_out]': [-4, 1, 3, 0, -7], 'massless': [5], 'sp': 10, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -4, 3, 0, -2], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5,10,2,2,3,3,1
1,"[0, 1]","[-1, -3, 1]","[1, 1, 5, -9, -9, 11]",2,6,"[{'[l,ν,s,m_in,m_out]': [-9, 1, 8, 0, -17], 'massless': [5, 11], 'sp': 16, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -9, 8, 0, -7], 'massless': [5, 11], 'sp': 16, 'sp_massless': []}]",11,2,2,2,2,3,10
2,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,6,"[{'[l,ν,s,m_in,m_out]': [-4, 9, -5, 1, 0], 'massless': [-11], 'sp': 22, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [9, -4, -5, 0, 14], 'massless': [1, -11], 'sp': 10, 'sp_massless': []}]",11,2,2,2,2,3,1
3,"[0, 1]","[-1, -4, 1]","[1, 1, 11, -16, -16, 19]",2,6,"[{'[l,ν,s,m_in,m_out]': [-16, 1, 15, 0, -31], 'massless': [11, 19], 'sp': 30, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -16, 15, 0, -14], 'massless': [11, 19], 'sp': 30, 'sp_massless': []}]",19,2,2,2,2,3,10
4,"[-3, 3]","[-3, -4]","[5, -9, -9, 16, 16, -19]",18,6,"[{'[l,ν,s,m_in,m_out]': [-9, 16, -7, 0, -2], 'massless': [5, -19], 'sp': 14, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [16, -9, -7, 0, 23], 'massless': [5, -19], 'sp': 14, 'sp_massless': []}]",19,2,2,2,2,3,10
5,"[0, 1]","[-1, -5, 1]","[1, 1, 19, -25, -25, 29]",2,6,"[{'[l,ν,s,m_in,m_out]': [-25, 1, 24, 0, -49], 'massless': [19, 29], 'sp': 48, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -25, 24, 0, -23], 'massless': [19, 29], 'sp': 48, 'sp_massless': []}]",29,2,2,2,2,3,10
6,"[-4, 4]","[-4, -5]","[11, -16, -16, 25, 25, -29]",32,6,"[{'[l,ν,s,m_in,m_out]': [-16, 25, -9, 0, -7], 'massless': [11, -29], 'sp': 18, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [25, -16, -9, 0, 34], 'massless': [11, -29], 'sp': 18, 'sp_massless': []}]",29,2,2,2,2,3,10


In [22]:
fl=fl.append( cl[ cl['n']==6 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [23]:
fl=fl.append(cl[ cl['n']==7 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [24]:
fl=fl.append(cl[ cl['n']==8 ].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True) )

In [25]:
fl=fl.append(cl[( (cl['n']==9) & (cl['D_msls']!=0) )].drop_duplicates(subset=['D_msls','rptd','nl','nν','nr','M_msls']).reset_index(drop=True))

In [26]:
fl=fl.reset_index(drop=True)

In [27]:
fl=fl.sort_values(['n','smax','D_msls','M_msls']).reset_index(drop=True)
fl

,l,k,solution,gcd,n,tree_level,smax,D_msls,rptd,nl,nν,nr,M_msls
0,"[-1, -2]","[-1, 2]","[1, 1, 1, -4, -4, 5]",1,6,"[{'[l,ν,s,m_in,m_out]': [-4, 1, 3, 0, -7], 'massless': [5], 'sp': 10, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -4, 3, 0, -2], 'massless': [5], 'sp': 10, 'sp_massless': []}]",5,10,2,2,3,3,1
1,"[1, -2]","[-4, 1]","[1, -4, -4, 9, 9, -11]",3,6,"[{'[l,ν,s,m_in,m_out]': [-4, 9, -5, 1, 0], 'massless': [-11], 'sp': 22, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [9, -4, -5, 0, 14], 'massless': [1, -11], 'sp': 10, 'sp_massless': []}]",11,2,2,2,2,3,1
2,"[0, 1]","[-1, -3, 1]","[1, 1, 5, -9, -9, 11]",2,6,"[{'[l,ν,s,m_in,m_out]': [-9, 1, 8, 0, -17], 'massless': [5, 11], 'sp': 16, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [1, -9, 8, 0, -7], 'massless': [5, 11], 'sp': 16, 'sp_massless': []}]",11,2,2,2,2,3,10
3,"[-1, 1]","[-1, 0, -1]","[1, 2, 2, -3, -3, -3, 4]",1,7,"[{'[l,ν,s,m_in,m_out]': [-3, 2, 1, 0, -4], 'massless': [1, 4], 'sp': 5, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, -3, 1, 1, 0], 'massless': [4], 'sp': 8, 'sp_massless': []}]",4,2,2,3,2,3,1
4,"[3, 1]","[-1, -5, 7]","[2, 2, -4, 7, -8, -8, 9]",1,7,"[{'[l,ν,s,m_in,m_out]': [2, -8, 6, -4, 0], 'massless': [7, 9], 'sp': 16, 'sp_massless': []}]",9,2,2,2,2,2,10
5,"[9, -6]","[-5, -10, 8]","[9, 9, -24, -24, -24, 25, 29]",45,7,"[{'[l,ν,s,m_in,m_out]': [-24, 9, 15, 0, -39], 'massless': [25, 29], 'sp': 54, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [9, -24, 15, 0, -6], 'massless': [25, 29], 'sp': 54, 'sp_massless': []}]",29,2,2,3,2,3,10
6,"[0, 1, -2]","[-1, -2, -4, 1]","[2, 2, 2, 2, -5, -5, -5, 7]",2,8,"[{'[l,ν,s,m_in,m_out]': [-5, 2, 3, 0, -8], 'massless': [7], 'sp': 14, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [2, -5, 3, 0, -1], 'massless': [7], 'sp': 14, 'sp_massless': []}]",7,10,2,3,4,3,1
7,"[-1, 0, 1]","[-4, -2, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",1,8,"[{'[l,ν,s,m_in,m_out]': [-7, -2, 9, 0, -16], 'massless': [1, 4, 5, 8], 'sp': 9, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [-2, -7, 9, 0, -11], 'massless': [1, 4, 5, 8], 'sp': 9, 'sp_massless': []}]",8,4,2,2,2,3,10
8,"[0, -3, -1]","[-3, -5, -6, -4]","[3, -4, -6, -6, 7, 7, 8, -9]",1,8,"[{'[l,ν,s,m_in,m_out]': [-6, 7, -1, 0, -5], 'massless': [3, -4, 8, -9], 'sp': 1, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [7, -6, -1, 8, 0], 'massless': [-9], 'sp': 18, 'sp_massless': []}]",9,4,2,2,2,3,1
9,"[1, -3, -2]","[-4, -9, -5, -3]","[1, 3, 3, 3, -5, -7, -7, 9]",1,8,"[{'[l,ν,s,m_in,m_out]': [-7, 3, 4, 0, -11], 'massless': [1, -5, 9], 'sp': 10, 'sp_massless': []}, {'[l,ν,s,m_in,m_out]': [3, -7, 4, 0, -1], 'massless': [1, -5, 9], 'sp': 10, 'sp_massless': []}]",9,10,2,2,3,3,3


In [28]:
kk=fl[['n','l','k','solution','gcd']].copy()
kkk=kk[['n','l','k','solution','gcd']]#Ref','DD','DM','XD','XM']]
def f(x):
    return  r'{}'.format(str(x).replace('[','(').replace(']',')'))

kkk.to_latex('solutions.tex',index=False,formatters=dict( [(k,f) for k in kk.columns ]) ,escape=False  )

In [29]:
cat solutions.tex

\begin{tabular}{rlllr}
\toprule
n &               l &                 k &                                 solution & gcd \\
\midrule
6 &        (-1, -2) &           (-1, 2) &                     (1, 1, 1, -4, -4, 5) &   1 \\
6 &         (1, -2) &           (-4, 1) &                   (1, -4, -4, 9, 9, -11) &   3 \\
6 &          (0, 1) &       (-1, -3, 1) &                    (1, 1, 5, -9, -9, 11) &   2 \\
7 &         (-1, 1) &       (-1, 0, -1) &                 (1, 2, 2, -3, -3, -3, 4) &   1 \\
7 &          (3, 1) &       (-1, -5, 7) &                 (2, 2, -4, 7, -8, -8, 9) &   1 \\
7 &         (9, -6) &      (-5, -10, 8) &            (9, 9, -24, -24, -24, 25, 29) &  45 \\
8 &      (0, 1, -2) &   (-1, -2, -4, 1) &              (2, 2, 2, 2, -5, -5, -5, 7) &   2 \\
8 &      (-1, 0, 1) &       (-4, -2, 2) &             (1, -2, -2, 4, 5, -7, -7, 8) &   1 \\
8 &     (0, -3, -1) &  (-3, -5, -6, -4) &             (3, -4, -6, -6, 7, 7, 8, -9) &   1 \\
8 &     (1, -3, -2) &  (-4, -9, -5, -3)